In [84]:
import pandas as pd
import spacy

In [85]:
df = pd.read_csv('./../data/train.csv')
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \r\n\r\nThat...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \r\n\r\nUmm, theres no actual article ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [86]:
df.duplicated().sum(), df.isnull().sum()

(0,
 id               0
 comment_text     0
 toxic            0
 severe_toxic     0
 obscene          0
 threat           0
 insult           0
 identity_hate    0
 dtype: int64)

In [87]:
df['sentiment'] = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].any(axis=1).astype(int)
df = df.drop(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"], axis=1)
df

,id,comment_text,sentiment
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0
159567,ffea4adeee384e90,You should be ashamed of yourself \r\n\r\nThat...,0
159568,ffee36eab5c267c9,"Spitzer \r\n\r\nUmm, theres no actual article ...",0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0


In [88]:
df['sentiment'].value_counts()

sentiment
0    143346
1     16225
Name: count, dtype: int64

In [89]:
nlp = spacy.load('en_core_web_sm')

In [90]:
def treat_comment(comment):
    spacy_comment = nlp(comment, disable=["parser", "tagger", "ner", "textcat"])
    treated_tokens = [w.text for w in spacy_comment if w.is_alpha and not w.is_stop]
    return " ".join(treated_tokens)

In [101]:
df_ones = df[df['sentiment'] == 1]
df_zeros = df[df['sentiment'] == 0]
df_zeros = df_zeros.sample(n=len(df[df['sentiment'] == 1]), random_state=42)
df = pd.concat([df_ones, df_zeros])

df['sentiment'].value_counts()

sentiment
1    16225
0    16225
Name: count, dtype: int64

In [102]:
df['comment_text'] = df['comment_text'].apply(treat_comment)

C:\Users\nbarb\AppData\Roaming\Python\Python312\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [103]:
Y = df["sentiment"]

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["comment_text"])
X, Y

(<32450x58857 sparse matrix of type '<class 'numpy.float64'>'
 	with 647917 stored elements in Compressed Sparse Row format>,
 6        1
 12       1
 16       1
 42       1
 43       1
         ..
 29806    0
 27165    0
 97660    0
 78629    0
 97994    0
 Name: sentiment, Length: 32450, dtype: int32)

In [104]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=42)

In [105]:
from sklearn.linear_model import LogisticRegression

In [106]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [107]:
model.score(X_train, Y_train)

0.9393516045527386

In [108]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [109]:
y_pred = model.predict(X_test)

In [110]:
accuracy_score(Y_test, y_pred), confusion_matrix(Y_test, y_pred)

(0.8905460372242081,
 array([[3759,  282],
        [ 606, 3466]], dtype=int64))

In [134]:
commentaire= 'you are a piece of shit'
test = treat_comment(commentaire)
vectorizer = TfidfVectorizer()
test = vectorizer.fit_transform([test])
y_pred = model.predict(test)


ValueError: X has 2 features, but LogisticRegression is expecting 58857 features as input.